# Week 9-2: Unsupervised Machine Learning

PR9-2: Great Outdoors wil graag weten welke producten vaak samen gekocht worden door klanten, door het bouwen van Frequent Itemsets met A-Priori-algoritme. Tip: merge eerst de tabellen 'product' en 'order_details' om een juiste tabel met brongegevens te krijgen waarop je het algoritme kan toepassen. 

* Pas waar nodig Dummy Encoding toe.
* Train het initiële algoritme.
* Experimenteer met meerdere support & confidence thresholds.
* De volgende webpagina's kun je als inspiratie gebruiken. Zij bevatten "codekapstokken" die uitleggen hoe je Frequent Itemsets moet maken en daarmee antwoord kunt geven op de vraag welke producten vaak samen gekocht worden.<br/>
  * <u>Geeks For Geeks</u> 
  * <u>Towards Data Science</u>

## Import libraries

Hier importeren we de libraries die we gaan gebruiken voor deze opdracht.

In [36]:
import pandas as pd
import sqlite3 as sql
from mlxtend.frequent_patterns import apriori, association_rules
import warnings as warn

warn.filterwarnings("ignore")

## Laden van de product tabel en de order_details tabel uit Great_Outdoors_Data_SQLite/go_sales.sqlite

We maken een connectie met de SQLite database, zodat we de juiste tabellen kunnen gebruiken voor het Apriori algoritme.

In [37]:
conn = sql.connect("Great_Outdoors_Data_SQLite/go_sales.sqlite")
conn

## Read the tables product, order_details and merge them

We lezen de product en order_details tabellen uit om ze daarna samen te kunnen voegen.

In [38]:
product = pd.read_sql_query("SELECT * FROM product", conn)
order_details = pd.read_sql_query("SELECT * FROM order_details", conn)

df = product.merge(order_details, on="PRODUCT_NUMBER")
df

,PRODUCT_NUMBER,INTRODUCTION_DATE,PRODUCT_TYPE_CODE,PRODUCTION_COST,MARGIN,PRODUCT_IMAGE,LANGUAGE,PRODUCT_NAME,DESCRIPTION,TRIAL888,ORDER_DETAIL_CODE,ORDER_NUMBER,QUANTITY,UNIT_COST,UNIT_PRICE,UNIT_SALE_PRICE,TRIAL879
0,1,15-2-2011,1,4,.33,P01CE1CG1.jpg,EN,TrailChef Water Bag,"Lightweight, collapsible bag to carry liquids ...",T,100085,8474,40,4.01,6.03,6.03,T
1,1,15-2-2011,1,4,.33,P01CE1CG1.jpg,EN,TrailChef Water Bag,"Lightweight, collapsible bag to carry liquids ...",T,100086,8470,40,4.01,6.03,6.03,T
2,1,15-2-2011,1,4,.33,P01CE1CG1.jpg,EN,TrailChef Water Bag,"Lightweight, collapsible bag to carry liquids ...",T,100087,8464,48,4.01,6.03,6.03,T
3,1,15-2-2011,1,4,.33,P01CE1CG1.jpg,EN,TrailChef Water Bag,"Lightweight, collapsible bag to carry liquids ...",T,100088,9254,66,4.01,6.03,6.03,T
4,1,15-2-2011,1,4,.33,P01CE1CG1.jpg,EN,TrailChef Water Bag,"Lightweight, collapsible bag to carry liquids ...",T,100089,8472,88,4.01,6.03,6.03,T
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43058,99,15-2-2011,17,2,.6,P99OP4FA17.jpg,EN,Aloe Relief,"Perfect for minor burns and sunburn, the aloe ...",T,98478,8210,30,1.97,4.94,4.94,T
43059,99,15-2-2011,17,2,.6,P99OP4FA17.jpg,EN,Aloe Relief,"Perfect for minor burns and sunburn, the aloe ...",T,99566,8246,20,2,5,3.95,T
43060,99,15-2-2011,17,2,.6,P99OP4FA17.jpg,EN,Aloe Relief,"Perfect for minor burns and sunburn, the aloe ...",T,99567,8239,18,2,5,5,T
43061,99,15-2-2011,17,2,.6,P99OP4FA17.jpg,EN,Aloe Relief,"Perfect for minor burns and sunburn, the aloe ...",T,99568,8270,26,2,5,5,T


## Destroy the useless TRIAL-columns

We vernietigen weer de nutteloze TRIAL-kolommen, omdat ze geen functie hebben.

In [39]:
df = df[df.columns.drop(list(df.filter(regex="TRIAL")))]
df

,PRODUCT_NUMBER,INTRODUCTION_DATE,PRODUCT_TYPE_CODE,PRODUCTION_COST,MARGIN,PRODUCT_IMAGE,LANGUAGE,PRODUCT_NAME,DESCRIPTION,ORDER_DETAIL_CODE,ORDER_NUMBER,QUANTITY,UNIT_COST,UNIT_PRICE,UNIT_SALE_PRICE
0,1,15-2-2011,1,4,.33,P01CE1CG1.jpg,EN,TrailChef Water Bag,"Lightweight, collapsible bag to carry liquids ...",100085,8474,40,4.01,6.03,6.03
1,1,15-2-2011,1,4,.33,P01CE1CG1.jpg,EN,TrailChef Water Bag,"Lightweight, collapsible bag to carry liquids ...",100086,8470,40,4.01,6.03,6.03
2,1,15-2-2011,1,4,.33,P01CE1CG1.jpg,EN,TrailChef Water Bag,"Lightweight, collapsible bag to carry liquids ...",100087,8464,48,4.01,6.03,6.03
3,1,15-2-2011,1,4,.33,P01CE1CG1.jpg,EN,TrailChef Water Bag,"Lightweight, collapsible bag to carry liquids ...",100088,9254,66,4.01,6.03,6.03
4,1,15-2-2011,1,4,.33,P01CE1CG1.jpg,EN,TrailChef Water Bag,"Lightweight, collapsible bag to carry liquids ...",100089,8472,88,4.01,6.03,6.03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43058,99,15-2-2011,17,2,.6,P99OP4FA17.jpg,EN,Aloe Relief,"Perfect for minor burns and sunburn, the aloe ...",98478,8210,30,1.97,4.94,4.94
43059,99,15-2-2011,17,2,.6,P99OP4FA17.jpg,EN,Aloe Relief,"Perfect for minor burns and sunburn, the aloe ...",99566,8246,20,2,5,3.95
43060,99,15-2-2011,17,2,.6,P99OP4FA17.jpg,EN,Aloe Relief,"Perfect for minor burns and sunburn, the aloe ...",99567,8239,18,2,5,5
43061,99,15-2-2011,17,2,.6,P99OP4FA17.jpg,EN,Aloe Relief,"Perfect for minor burns and sunburn, the aloe ...",99568,8270,26,2,5,5


## Stuff that is here

Dit staat hier om te staan.

In [40]:
# De kolommen, zonder TRIAL: 
# product: PRODUCT_NUMBER, INTRODUCTION_DATE, PRODUCT_TYPE_CODE, PRODUCTION_COST, MARGIN, PRODUCT_IMAGE, LANGUAGE, PRODUCT_NAME, DESCRIPTION
# order_details: ORDER_DETAIL_CODE, ORDER_NUMBER, QUANTITY, UNIT_COST, UNIT_PRICE, UNIT_SALE_PRICE

# Dummy encoding toepassen op de data

## We drop columns that we don't need for the A-Priori algorithm

We laten kolommen vallen die we niet meer nodig hebben om het A-Priori algoritme te kunnen trainen.

In [41]:
# These are the candidates: 
# product: INTRODUCTION_DATE, PRODUCTION_COST, MARGIN, LANGUAGE, PRODUCT_NAME, DESCRIPTION 
# order_details: ORDER_DETAIL_CODE, ORDER_NUMBER, UNIT_COST, UNIT_PRICE, UNIT_SALE_PRICE

# Remove some columns
df = df.drop(["PRODUCT_NUMBER", "PRODUCT_TYPE_CODE", "PRODUCT_IMAGE", "QUANTITY"], axis=1)
df

,INTRODUCTION_DATE,PRODUCTION_COST,MARGIN,LANGUAGE,PRODUCT_NAME,DESCRIPTION,ORDER_DETAIL_CODE,ORDER_NUMBER,UNIT_COST,UNIT_PRICE,UNIT_SALE_PRICE
0,15-2-2011,4,.33,EN,TrailChef Water Bag,"Lightweight, collapsible bag to carry liquids ...",100085,8474,4.01,6.03,6.03
1,15-2-2011,4,.33,EN,TrailChef Water Bag,"Lightweight, collapsible bag to carry liquids ...",100086,8470,4.01,6.03,6.03
2,15-2-2011,4,.33,EN,TrailChef Water Bag,"Lightweight, collapsible bag to carry liquids ...",100087,8464,4.01,6.03,6.03
3,15-2-2011,4,.33,EN,TrailChef Water Bag,"Lightweight, collapsible bag to carry liquids ...",100088,9254,4.01,6.03,6.03
4,15-2-2011,4,.33,EN,TrailChef Water Bag,"Lightweight, collapsible bag to carry liquids ...",100089,8472,4.01,6.03,6.03
...,...,...,...,...,...,...,...,...,...,...,...
43058,15-2-2011,2,.6,EN,Aloe Relief,"Perfect for minor burns and sunburn, the aloe ...",98478,8210,1.97,4.94,4.94
43059,15-2-2011,2,.6,EN,Aloe Relief,"Perfect for minor burns and sunburn, the aloe ...",99566,8246,2,5,3.95
43060,15-2-2011,2,.6,EN,Aloe Relief,"Perfect for minor burns and sunburn, the aloe ...",99567,8239,2,5,5
43061,15-2-2011,2,.6,EN,Aloe Relief,"Perfect for minor burns and sunburn, the aloe ...",99568,8270,2,5,5


## All columns of the `df` DataFrame will be One-hot (Dummy) encoded

Nu zullen alle kolommen van het `df` DataFrame One-hot encoded worden en dus heel dom zijn, nou ja meer slim. We maken dummies.

In [42]:
# Dummy encode all columns
df = pd.get_dummies(df)
df

,INTRODUCTION_DATE_10-1-2020,INTRODUCTION_DATE_10-1-2021,INTRODUCTION_DATE_10-12-2019,INTRODUCTION_DATE_12-6-2015,INTRODUCTION_DATE_13-1-2020,INTRODUCTION_DATE_15-1-2020,INTRODUCTION_DATE_15-12-2019,INTRODUCTION_DATE_15-2-2011,INTRODUCTION_DATE_18-12-2019,INTRODUCTION_DATE_26-10-2016,...,UNIT_SALE_PRICE_99,UNIT_SALE_PRICE_99.23,UNIT_SALE_PRICE_99.3,UNIT_SALE_PRICE_99.75,UNIT_SALE_PRICE_99.76,UNIT_SALE_PRICE_99.9,UNIT_SALE_PRICE_990.03,UNIT_SALE_PRICE_992.65,UNIT_SALE_PRICE_993.53,UNIT_SALE_PRICE_997.8
0,False,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43058,False,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
43059,False,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
43060,False,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
43061,False,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False


## Trainen van A-Priori algoritme op dataframe `df`

Hier zullen we de frequent itemsets trainen van het A-Priori algoritme op het dataframe `df`.

In [43]:
# Trainen van A-Priori algoritme op dataframe `df` om te zien welke producten vaak samen verkocht worden aan klanten. Hieronder de code:
frequent_itemsets = apriori(df, min_support=0.02, use_colnames=True)
#frequent_itemsets = apriori(df, min_support=0.5, use_colnames=True, max_len=2, verbose=1, low_memory=True) # replace later

# Regels genereren
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=1.0)
#rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.0, support_only=False) # replace later
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(INTRODUCTION_DATE_10-1-2021),(LANGUAGE_EN),0.165827,1.000000,0.165827,1.0,1.000000,0.000000,inf,0.000000
1,(INTRODUCTION_DATE_10-12-2019),(LANGUAGE_EN),0.034043,1.000000,0.034043,1.0,1.000000,0.000000,inf,0.000000
2,(INTRODUCTION_DATE_15-12-2019),(LANGUAGE_EN),0.026055,1.000000,0.026055,1.0,1.000000,0.000000,inf,0.000000
3,(PRODUCTION_COST_3),(INTRODUCTION_DATE_15-2-2011),0.023315,0.378701,0.023315,1.0,2.640606,0.014485,inf,0.636130
4,(MARGIN_.67),(INTRODUCTION_DATE_15-2-2011),0.020807,0.378701,0.020807,1.0,2.640606,0.012927,inf,0.634501
...,...,...,...,...,...,...,...,...,...,...
92,"(MARGIN_.5, PRODUCTION_COST_3, INTRODUCTION_DA...",(LANGUAGE_EN),0.023315,1.000000,0.023315,1.0,1.000000,0.000000,inf,0.000000
93,"(LANGUAGE_EN, PRODUCTION_COST_3)","(MARGIN_.5, INTRODUCTION_DATE_15-2-2011)",0.023315,0.034136,0.023315,1.0,29.294558,0.022519,inf,0.988920
94,"(MARGIN_.5, PRODUCTION_COST_3)","(LANGUAGE_EN, INTRODUCTION_DATE_15-2-2011)",0.023315,0.378701,0.023315,1.0,2.640606,0.014485,inf,0.636130
95,"(PRODUCTION_COST_3, INTRODUCTION_DATE_15-2-2011)","(LANGUAGE_EN, MARGIN_.5)",0.023315,0.194622,0.023315,1.0,5.138170,0.018777,inf,0.824604
